In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
dataset = pd.read_csv('./london_merged.csv')

# Opis danych

#### Zbiór danych zawiera następujące kolumny:
- `timestamp` - znacznik czasowy
- `cnt` - liczba nowych wypożyczeń rowerów
- `t1` - temperatura w stopniach Celsjusza
- `t2` - temperatura odczuwalna w stopniach Celsjusza
- `hum` - wilgotność powietrza (skala procentowa)
- `wind_speed` - pędkość wiatru w $\frac{km}{h}$
- `weather_code` - kategoria pogody w kategoriach:
    - 1 - Czyste niebo / czyste z nielicznymi zjawiskami, jak lekka mgła itp.
    - 2 - Obecne chmury
    - 3 - Promienie przedzierające się przez chmury (chmury pokrywające 60-90 % nieba)
    - 4 - Pochmurno
    - 7 - Deszcz/mżawka
    - 10 - Deszcz z burzą
    - 26 - Opad śniegu
    - 94 - mgła lodowa
- `is_holiday` - oznacza święto/wakacje w danym dniu (1  oznacza święto, zero jego brak)
- `is_weekend` - oznacza dzień zaliczany do weekendu (1 weekend, 0 jego brak)
- `season` - określenie pory roku:
    - 0 - wosna
    - 1 - lato
    - 2 - jesień
    - 3 - zima

#### Przykładowy wycinek z danych

In [3]:
dataset.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


#### Liczność danych (kolumny x wiersze)

In [4]:
f"{dataset.shape[1]} kolumn x {dataset.shape[0]} wierszy"

'10 kolumn x 17414 wierszy'

# Wstępna charakterystyka i walidacja danych

Wstępna charakterystyka danych może od samego początku wskazać błędy w danych. Wnioskowanie można przeprowadzić ze względu na naturalne ograniczenia parametrów:
- `temperatura` w przedziale <-50, 50> &deg;C
- `wilgotność` powietza w przedziale <0,100> %
- `prędkość` wiatru w przedziale <0, 250> $\frac{km}{h}$

Ich poprawność wg w.w kryteriów można określić za pomocą tabeli podsumowania danych na podstawie wartości `min` oraz `max`.

In [5]:
dataset.describe()

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
count,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000
mean,1143.101642,12.468091,11.520836,72.324954,15.913063,2.722752,0.022051,0.285403,1.492075
std,1085.108068,5.571818,6.615145,14.313186,7.894570,2.341163,0.146854,0.451619,1.118911
min,0.000000,-1.500000,-6.000000,20.500000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,257.000000,8.000000,6.000000,63.000000,10.000000,1.000000,0.000000,0.000000,0.000000
50%,844.000000,12.500000,12.500000,74.500000,15.000000,2.000000,0.000000,0.000000,1.000000
75%,1671.750000,16.000000,16.000000,83.000000,20.500000,3.000000,0.000000,1.000000,2.000000
max,7860.000000,34.000000,34.000000,100.000000,56.500000,26.000000,1.000000,1.000000,3.000000


Pozostałe atrybuty to wartości dyskretne ze ściśle określonych zbiorów:
- `weather_code` $\in \{1,2,3,4,7,10,26,94\}$
- `is_holiday` $\in \{0,1\}$
- `is_weekend` $\in \{0,1\}$
- `season` $\in \{0,1,2,3\}$

Ich poprawność można określić poprzez sprawdzenie, czy wartości zawierają się w zadanych zbiorach.

In [6]:
weather_code_validation_set = (1, 2, 3, 4, 7, 10, 26, 94)
is_holiday_validation_set = (0, 1)
is_weekend_validation_set = (0, 1)
season_validation_set = (0, 1, 2, 3)

validation_sets = {'weather_code': weather_code_validation_set, 'is_holiday': is_holiday_validation_set,
                   'is_weekend': is_weekend_validation_set, 'season': season_validation_set}

dataset[validation_sets.keys()].isin(validation_sets).all(axis=0)



weather_code    True
is_holiday      True
is_weekend      True
season          True
dtype: bool

#### Statystyka danych brakujących (NULL)

In [7]:
dataset.isna().any(axis=0)

timestamp       False
cnt             False
t1              False
t2              False
hum             False
wind_speed      False
weather_code    False
is_holiday      False
is_weekend      False
season          False
dtype: bool

Test wykazuje brak danych o wartościach nieokreślonych

#### Wnioski
Wstępne testy wykazały poprawność formalną danych, spełniają określone założenia, tj. ich wartości mieszczą się w zadanych przedziałach, a dla wartości dyskretnych - w zbiorach.

# Badanie wartości nietypowych

### Metoda 3 - sigma

In [10]:
outliers_3_score = np.where(
    np.abs(stats.zscore(dataset[dataset.keys().drop('timestamp')])) > 3)
outliers_3_score

outliered_dataset = dataset.iloc[outliers_3_score[0]]

outliered_dataset

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
138,2015-01-09 19:00:00,1094,15.5,15.5,77.0,41.0,7.0,0.0,0.0,3.0
139,2015-01-09 20:00:00,686,15.0,15.0,72.0,44.0,3.0,0.0,0.0,3.0
140,2015-01-09 21:00:00,443,15.0,15.0,68.0,41.0,3.0,0.0,0.0,3.0
141,2015-01-09 22:00:00,464,15.0,15.0,68.0,40.0,3.0,0.0,0.0,3.0
142,2015-01-09 23:00:00,420,14.0,14.0,77.0,41.5,4.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17385,2017-01-02 19:00:00,433,3.0,0.0,81.0,11.0,1.0,1.0,0.0,3.0
17386,2017-01-02 20:00:00,334,3.0,0.0,75.0,13.0,1.0,1.0,0.0,3.0
17387,2017-01-02 21:00:00,233,2.5,-0.5,78.0,11.0,1.0,1.0,0.0,3.0
17388,2017-01-02 22:00:00,201,2.0,-1.0,81.0,10.0,1.0,1.0,0.0,3.0
